# Supercompressible: optimization 3d

This notebook intends to show the optimization process to maximize the absorbed energy in the 3D case. 

So, let's start by importing the required libraries:

In [1]:
# standard library
import pickle

# third-party
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.gaussian_process.kernels import Matern
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from scipy.optimize import minimize
from matplotlib.colors import ListedColormap

ModuleNotFoundError: No module named 'sklearn'

Now, let's import the data.

In [ ]:
# get pandas frame
filename = 'DoE_results.pkl'
with open(filename, 'rb') as file:
    data = pickle.load(file)
points = data['points']
print('variables:', [col for col in points.columns])

# get number of inputs
n_inputs = len(points.columns) - 3

# get X data
X = points.iloc[:,range(n_inputs)].values
X_full = points.iloc[:,range(n_inputs)].values

# get energy data
var_name = 'energy'
y = points.loc[:, var_name].values

# get coilable data
class_name = 'coilable'
y_class = points.loc[:, class_name].values

print('X:', X)
print('X shape:', np.shape(X))
print('y:', y)
print('y shape:', np.shape(y))
print('y_class:', y_class)
print('y_class shape:', np.shape(y_class))

The variables ```X``` and ```y``` contain all the data. Nevertheless, there's ```NaN``` values that cannot be considered.

In [ ]:
for yy in np.unique(y_class):
    print('Class %i: %i' % (int(yy), np.sum(y_class==yy)))

In [ ]:
# missing indices
indices = pd.notnull(points.loc[:, var_name]).values

# get matching energy and coiling data
y = points.loc[indices, var_name].values
# y_class = points.loc[indices, class_name].values

# get X data
X = points.iloc[indices,range(n_inputs)].values

print('X:', X)
print('X shape:', np.shape(X))
print('y:', y)
print('y shape:', np.shape(y))
print('y_class:', y_class)
print('y_class shape:', np.shape(y_class))

Let's now clean the data.

In [ ]:
def perform_cleaning(X, y, n_std_cleaning):
    
    # compute threshold
    y_mean, y_std = np.mean(y), np.std(y)
    y_thresh = y_mean + n_std_cleaning * y_std

    # indices
    print(y_thresh)
    indices = np.where(y < y_thresh)
    n_dismissed_points = len(y) - len(indices[0])
    print('Dismissed points:', n_dismissed_points)

    # X and y
    y = y[indices]
    X = X[indices]
    print(np.shape(y))
    print(np.shape(X))

    return X, y, n_dismissed_points

In [ ]:
n_std_cleaning = 5  # dismiss points that fall outside this range

n_dismissed_points = 1
while n_dismissed_points > 0:
    X, y, n_dismissed_points = perform_cleaning(X, y, n_std_cleaning)

Methods to create and evaluate the regression and classification models.

In [ ]:
def make_gp_regression(X_train, y_train, kernel):
    
    # train model
    model = GaussianProcessRegressor(kernel=kernel, alpha=0.1**2, 
                                   n_restarts_optimizer=0)
    model.fit(X_train, y_train)
    
    return model

In [ ]:
def split_data(X, y, train_size):
    
    # split data
    test_size = 1 - train_size
    indices = range(len(y))
    X_train = X[indices[:-int(round(len(indices) * test_size))]]
    X_test = X[indices[-int(round(len(indices) * test_size)):]]
    y_train = y[indices[:-int(round(len(indices) * test_size))]]
    y_test = y[indices[-int(round(len(indices) * test_size)):]]
    
    return (X_train, X_test, y_train, y_test)

In [ ]:
def evaluate_model(model, X_test, y_mean_test, n_train):
    
    # predict test
    y_mean_pred = model.predict(X_test, return_std=False)

    # error metrics
    mse = mean_squared_error(y_mean_test, y_mean_pred)
    r2 = r2_score(y_mean_test, y_mean_pred)
    expl_var = explained_variance_score(y_mean_test, y_mean_pred)
    print("The mean squared error is %0.3e" % mse)
    print("The R2 score is %0.3f" % r2)
    print("The explained variance score is %0.3f" % expl_var)

    # plot predicted vs observed
    plt.figure()
    plt.plot(y_mean_test, y_mean_pred, 'o')
    plt.plot([np.min(y_mean_test), np.max(y_mean_test)], [np.min(y_mean_test), np.max(y_mean_test)], 'r-')
    plt.title('Posterior kernel: %s\n$n_{train} = %i$, $R^{2} = %0.3f$, $ MSE = %0.3e$' % (model.kernel_, n_train, r2, mse))
    plt.ylabel("Predicted")
    plt.xlabel("Observed")
    plt.show()

In [ ]:
def evaluate_clf_model(clf, train_data, test_data, metrics=((accuracy_score, 'Accuracy classification score'),), print_results=True):
    
    # make predictions
    y_train_predicted = clf.predict(train_data[0])
    y_test_predicted = clf.predict(test_data[0])
    
    metric_results = []
    for Imetric in range(np.shape(metrics)[0]):
        metric = metrics[Imetric][0] # select metric used
        metric_results.append([metric(train_data[1], y_train_predicted), metric(test_data[1], y_test_predicted)])
        
        if print_results:
            print("Train", metrics[Imetric][1],"for ML model:", metric_results[-1][0])
            print("Test", metrics[Imetric][1],"for ML model:", metric_results[-1][1])
        
    return metric_results

Next, we will prepare the classification.

In [ ]:
# choose train size
train_size = .8
(X_train, X_test, y_train, y_test) = split_data(X_full, y_class, train_size)
n_train = len(X_train)

# scale data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# train
clf = SVC(kernel='rbf').fit(X_train_scaled, y_train)

# evaluate model
evaluate_clf_model(clf, (X_train_scaled, y_train), (X_test_scaled, y_test));

In [ ]:
xx = [[0.04276196, 0.41448975, 0.69628906], # best in full dataset
      [0.03954443, 0.51306152, 0.65351563],  # best in full dataset (N=3000)
      [0.0337832, 0.32080078, 0.7578125],  # best in full dataset (N=1000)
      [0.03710181, 0.26312256, 0.77441406], # best in the dataset after classification with N=10000
      [0.02876318, 0.31774902, 0.72851562], # best in the dataset after classification with N=5000
      [0.0337832,  0.32080078, 0.7578125], # best in the dataset after classification with N=3000 and N=1000
      [0.04, 0.417, 0.8], # best found after after classification and regression with N=10000
      [0.038, 0.32, 0.8], # best found after after classification and regression with N=5000
      [0.037, 0.32, 0.8], # best found after after classification and regression with N=3000
      [0.037, 0.4, 0.8], # best found after after classification and regression with N=1000
      ]
yy = clf.predict(scaler.transform(xx))

print(yy)

In [ ]:
# choose train size
train_size = .1
(X_train, X_test, y_train, y_test) = split_data(X, y, train_size)
n_train = len(X_train)

# scale data
# scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# train and predict
kernel = 1.0 * Matern(length_scale=1.0, nu=2.5, length_scale_bounds=(1e-1, 10.0))
model = make_gp_regression(X_train_scaled, y_train, kernel)

# evaluate model
# evaluate_model(model, X_test_scaled, y_test, n_train)

In [ ]:
xx = [[0.04276196, 0.41448975, 0.69628906]]
yy = clf.predict(scaler.transform(xx))
yyen = model.predict(scaler.transform(xx))

print(xx)
print(yy)
print(yyen)

We now have our classification and regression model ready. We can use these functions to optimize for the absorbed energy while making sure that the coilability is favorable (class 1). We can do this by penalizing the other classes. A similar action can be undertaken when the input point does not fall within the bounds of the parameter space.

In [ ]:
variables = ['ratio_d', 'ratio_pitch', 'ratio_top_diameter']
bounds = [data['doe_variables'][name] for name in variables]

# print(bounds)
# print()

# def validitycheck(x):
#     res = 0
#     for i in range(3):
#         a = bounds[i][0] <= x[0][i]
#         b = x[0][i] <= bounds[i][1]
#         c = clf.predict(scaler.transform(x)) == 1
#         res += a + b + c
#     if res == 9:
#         return True
#     else:
#         return False

# print(validitycheck([[0.04, 0.417, 0.8]]))
# print()

def objfun(x):
#     penalty = 1e5
    x = [list(x)]
#     print(x)
#     print(model.predict(scaler.transform(x)))
#     print(clf.predict(scaler.transform(x)))
#     print()
    if np.rint(clf.predict(scaler.transform(x))) == 1:
        output = -model.predict(scaler.transform(x))
    else:
        output = model.predict(scaler.transform(x))
#         output = penalty
#     print(-output)
#     print()
    return output

x0 = [[0.04, 0.417, 0.8]]
optval = minimize(objfun, x0=x0, method='Nelder-Mead')

print(optval)
print()
print(clf.predict(scaler.transform([list(optval.x)])))